## Installed packages

In [ ]:
#Import packages
import bisect
import datetime
import folium
import functools
import geopandas as gpd
import io
import logging
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt, dates
import networkx as nx  
import numpy as np
import os
import pandas as pd
import pickle
import pyproj
import requests
import scipy as sc
import shapely.geometry
from shapely.geometry import Point,Polygon,MultiPolygon
from shapely.ops import transform
import shapely.wkt
import time as timpie
import xarray as xr
import scipy as sc
import yaml
import pytz

#Import packages OpenTNSim
from opentnsim import core
from opentnsim import plot
from opentnsim import model
from opentnsim import import_hydrodynamic_dataset
from opentnsim import vessel_traffic_service
from opentnsim import port
from opentnsim import lock
from opentnsim import vessel as vessel_
from opentnsim import waterway
from opentnsim import output
from opentnsim import tidal_window_constructor
from opentnsim import rule_constructor

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
geod = pyproj.Geod(ellps="WGS84")
pd.set_option('display.max_rows', 500)

%load_ext autoreload
%autoreload 2

## Set paths

In [ ]:
path = os.getcwd()
path = path.split('\\03_Simulation')[0]

## Set default colors

In [ ]:
color_lijnA = (0/255,152/255,51/255)
color_lijnB = (253/255,223/255,1/255)
color_lijnC = (231/255,33/255,24/255)
color_lijnD = (52/255,180/255,229/255)
color_lijnE = (2/255,58/255,141/255)
color_lijnF = (212/255,103/255,160/255)
colors = [color_lijnA,color_lijnB,color_lijnC,color_lijnD,color_lijnE,color_lijnF]

## Some functions

In [ ]:
def calculate_additional_waiting_time(origin_destination_matrix, trip_id1,trip_id2):
    """ 
    Function that calculates the waiting time if another vessel was observed to be having priority

    Parameters
    ----------
    origin_destination_matrix: origin-destination matrix of the vessels as pandas dataframe
    trip_id1: trip ID as string of vessel that was observed to have additional waiting time
    trip_id2: trip ID as string of vessel that was observed to be having priority over the other vessel

    :returns: dictionary with vessel name as name and delay as value
    """
    
    arrival_time1 = origin_destination_matrix[origin_destination_matrix.trip_id == trip_id1].iloc[0].arrival
    arrival_time2 = origin_destination_matrix[origin_destination_matrix.trip_id == trip_id2].iloc[0].arrival
    delay = arrival_time2 - arrival_time1
    return {trip_id1:delay}

def create_vessel(Vessel,env,name,origin,destination,next_destination,beam,length,draught,delta_draught,berthing_time,unloading_time,turning_time,arrival_time,terminal_of_call,berth_of_call,additional_waiting_time,bound='inbound',height=0.,ukc=0.,max_cross_current=0.):
    """ 
    Function that creates a vessel agent in the nautical traffic model

    Parameters
    ----------
    Vessel: type object element that includes the mixin objects that symbolize a vessel agent
    env: simpy environment
    name: name of the vessel as a string
    origin: origin node of the network as a string
    destination: destination node of the network as a string
    next_destination: list of next destination nodes of the network as a string
    beam: beam of the vessel in meters as a float
    length: length of the vessel in meters as a float
    draught: draught of the vessel in meters as a float
    delta_draught: list of draught changes of the vessel in meters as a float in sequence of terminal visits
    berthing_time: berthing time of the vessel as pandas timestamp
    unloading_time: list of unloading times of the vessel as pandas timestamps in sequence of terminal visits
    turning_time: list of turning times of the vessel as pandas timestamps in sequence of harbour basin visits
    arrival_time: rrival time of vessel as pandas timestamp
    terminal_of_call: list of terminal names as string in sequence of terminal visits
    berth_of_call: list of berth names as string in sequence of terminal visits
    additional_waiting_time: additional waiting time as pandas timedelta
    bound: 'inbound' or 'outbound'
    height: height of the vessel in meters as a float
    ukc: extra required under keel clearance of the vessel in meters as a float
    max_cross_current: additional maximum allowable cross-current velocity of the vessel in meters per second as a float

    :returns: vessel agent
    """
    
    vessel_input = { "name":name,
                     "origin":origin,
                     "destination":destination,
                     "next_destination":next_destination,
                     "env":env,
                     "type":'Tanker',
                     "B":beam,
                     "L":length,
                     "T": draught,
                     "H":height,
                     "t_berthing":berthing_time.total_seconds(),
                     "t_(un)loading":[time.total_seconds() for time in unloading_time],
                     "t_turning":[time.total_seconds() for time in turning_time],
                     "ukc":ukc,
                     "v":np.NaN,
                     "terminal_of_call": terminal_of_call,
                     "berth_of_call": berth_of_call,
                     "(un)loading": delta_draught,
                     "max_waiting_time":datetime.timedelta(days=10).total_seconds(),
                     "max_cross_current":max_cross_current,
                     "arrival_time":arrival_time,
                     "arrival_delay":arrival_time,
                     "priority": 0,
                     "additional_waiting_time": additional_waiting_time/np.timedelta64(1, 's'),
                     "bound":bound,
                     "priority":False}
    
    created_vessel = Vessel(**vessel_input)
    return created_vessel

## Simulation settings

In [ ]:
simulation_start = datetime.datetime(2019,1,1,0,0,0).replace(tzinfo=pytz.utc)
simulation_stop = datetime.datetime(2020,1,1,0,0,0).replace(tzinfo=pytz.utc)

## Import graph

In [ ]:
with open(path+"\\03_Simulation\\01_Input_data\\01_Geospatial_data\\network"+"\\PortNetwork.pickle", 'rb') as f:
    FG = pickle.load(f)

## Open hydrodynamic data

In [ ]:
#Open analyzed and structured hydrodynamic data
hydrodynamic_data = xr.open_dataset(path+"\\03_Simulation\\01_Input_data\\02_Hydrodynamic_data\\+'hydrodynamic_data_PoR_stations.nc")

In [ ]:
#Overwrite MBLs (Change MBLs here)
node_list = hydrodynamic_data['STATION'].values
hydrodynamic_data['MBL'] = xr.DataArray(23*np.ones(len(node_list)),coords={'STATION':node_list})
for index,node in enumerate(node_list):
    if node in ['8866305','8864266','8862925','8864465','S14716_B','S14716_A','8860845']:
        hydrodynamic_data['MBL'][index] = 16.2
    elif node in ['8867547','8867980','8866999']:
        hydrodynamic_data['MBL'][index] = 15.9
    elif node in ['8866859']:
        hydrodynamic_data['MBL'][index] = 15.9
    elif node in ['anchorage','8866969']:
        hydrodynamic_data['MBL'][index] = 23.8
    else:
        hydrodynamic_data['MBL'][index] = 16.4
        
#Possibility to change bed level over time
new_MBLs = np.empty((len(hydrodynamic_data['STATION']),len(hydrodynamic_data['TIME'])))
for station,MBL in enumerate(hydrodynamic_data['MBL'].values):
    for time,_ in enumerate(hydrodynamic_data['TIME'].values):
        new_MBLs[station][time] = MBL
        
new_MBLs = xr.DataArray(new_MBLs,dims=['STATION','TIME'],coords=dict(STATION=hydrodynamic_data['STATION'],
                                                                     TIME=hydrodynamic_data['TIME']))

hydrodynamic_data['MBL'] = new_MBLs

In [ ]:
#Read tidal period data
for tide_type,dim_name in zip(['Vertical tidal periods','Horizontal tidal periods'],['VERTICALTIDES','HORIZONTALTIDES']):
    data = np.empty(hydrodynamic_data[tide_type].values.shape[:2], dtype=[('a','datetime64[ns]'),('b',object)])
    tide_info = hydrodynamic_data[tide_type].values
    for station_index,_ in enumerate(hydrodynamic_data.STATION.values):
        for tide_index in hydrodynamic_data[dim_name].values:
            time = tide_info[station_index][tide_index][0]
            tide = tide_info[station_index][tide_index][1]
            if time == 'nan':
                new_time = 'NaT'
            else:
                new_time = time
            data[station_index][tide_index] = (np.datetime64(new_time),tide)
    
    final_data = []
    for i,_ in enumerate(data):
        final_data.append([])
        final_data[-1].append([])
        for j,_ in enumerate(data[0]):
            final_data[-1][-1].append([data[i][j][0],data[i][j][1]])
    hydrodynamic_data[tide_type].data = np.array(final_data).squeeze()

## Add vessel speeds

In [ ]:
with open(path+"\\03_Simulation\\01_Input_data\\03_Vessels\\vessel_speed_dataframe.pickle", "rb") as input_file:
    vessel_speed_dataframe = pickle.load(input_file)

In [ ]:
#Add missing vessel speeds and convert to xarray
missing_vessel_speeds = pd.DataFrame([0.1,0.1,5,5],columns=['average_speed'],index=[('anchorage','8866969'),('8866969','anchorage'),('B17838816_B', 'B17838816_A'),('B17838816_A', 'B17838816_B')])
missing_vessel_speeds.index.name = 'edge'
vessel_speed_dataframe = pd.concat([vessel_speed_dataframe,missing_vessel_speeds],axis=0)
vessel_speed_data = vessel_speed_dataframe.to_xarray()

## Create Simulation

In [ ]:
sim = model.Simulation(graph = FG,
                       simulation_start=simulation_start,
                       simulation_stop=simulation_stop,
                       hydrodynamic_data = hydrodynamic_data,
                       vessel_speed_data = vessel_speed_data)
env = sim.environment

## Import vessels

In [ ]:
with open(path+"\\03_Simulation\\01_Input_data\\03_Vessels\\origin_destination_PoR.pickle", "rb") as input_file:
    origin_destination_matrix = pickle.load(input_file)

In [ ]:
#Some corrections to the matrix
origin_destination_matrix['berth_node'] = '8866859'
origin_destination_matrix = origin_destination_matrix.sort_values('arrival')
origin_destination_matrix = origin_destination_matrix.reset_index(drop=True)
origin_destination_matrix.loc[69,'(un)loading time'] = origin_destination_matrix.loc[69,'(un)loading time']/2
origin_destination_matrix.loc[69,'destination_node'] = '8868178'
origin_destination_matrix.loc[144,'berth_of_call'] = 'Koole_Buiten10'
origin_destination_matrix.loc[259,'berth_of_call'] = 'Koole_Kade_H'
origin_destination_matrix.loc[460,'arrival'] = pd.Timestamp('2019-12-08 05:20:00+0000', tz=pytz.utc)
for index in origin_destination_matrix[origin_destination_matrix.origin_node == '8866859'].index:
    origin_destination_matrix.loc[index,'origin_node'] = '8866999'

In [ ]:
#Define arrival times, departure times and berths of calls for the nautical traffic model
origin_destination_matrix['arrival_time'] = origin_destination_matrix['arrival']+origin_destination_matrix['waiting_time_in_anchorage']
origin_destination_matrix['departure_time'] = origin_destination_matrix['arrival_time']+origin_destination_matrix['(un)loading time']
origin_destination_matrix['berth_of_call'] = [berth.split('Koole_')[1] for berth in origin_destination_matrix['berth_of_call']]
corrected_origin_destination_matrix = origin_destination_matrix.copy()
merge_locs = {}
for column in origin_destination_matrix.columns:
    origin_destination_matrix[column] = origin_destination_matrix[column].astype('object')
for name in list(dict.fromkeys(origin_destination_matrix.name)):
    df_ship = origin_destination_matrix[origin_destination_matrix.name == name]
    if len(df_ship) > 1:
        remove_indexes = []
        trip_merge = False
        for (_,prev_row),(_,next_row) in zip(df_ship.iloc[:-1].iterrows(),df_ship.iloc[1:].iterrows()):
            if next_row.arrival_time - prev_row.departure_time < pd.Timedelta(2,'h') and next_row.origin_node in ['8866999','8866859']:
                if not trip_merge:
                    merge_loc = prev_row.name
                    merge_trip_id = prev_row.trip_id
                    merge_locs[merge_trip_id] = []
                    trip_merge = True
                if trip_merge: 
                    remove_indexes.append(next_row.name)
                    merge_locs[merge_trip_id].append(next_row.trip_id)
                    for column in ['(un)loading','berth_of_call','(un)loading time','berth_node','destination_node','turning_time']:
                        origin_destination_matrix.at[merge_loc,column] = list(np.append(np.array(origin_destination_matrix.loc[merge_loc,column]),next_row[column]))   
            elif trip_merge:        
                trip_merge = False
        origin_destination_matrix = origin_destination_matrix.drop(remove_indexes)     

In [ ]:
#Define other parameters for the nautical traffic model
for column in ['(un)loading','berth_of_call','turning_time','(un)loading time','berth_node','destination_node']:
    values = []
    for value in origin_destination_matrix[column].to_numpy():
        if not isinstance(value,list):
            values.append([value])
        else:
            values.append(value)
    origin_destination_matrix[column] = values

In [ ]:
#Some corrections to these numbers
for index in [12,109,156,175]:
    origin_destination_matrix.loc[index,'(un)loading'] = [np.max(origin_destination_matrix.loc[index]['(un)loading'])]
    origin_destination_matrix.loc[index,'berth_of_call'] = [origin_destination_matrix.loc[index]['berth_of_call'][0]]
    origin_destination_matrix.loc[index,'turning_time'] = [origin_destination_matrix.loc[index]['turning_time'][0]]
    origin_destination_matrix.loc[index,'(un)loading time'] = [np.sum(origin_destination_matrix.loc[index]['(un)loading time'])]
    origin_destination_matrix.loc[index,'destination_node'] = [origin_destination_matrix.loc[index]['destination_node'][-1]]

In [ ]:
#Calculates additional waiting time of vessels
additional_waiting_times = {}
additional_waiting_times = {**calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-3417_0','testschip-2718_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-4407_0','testschip-4157_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-1186_0','testschip-3491_1'), 
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-8014_0','testschip-1635_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-9416_0','testschip-4025_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-3005_0','testschip-4815_1'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-12542_0','testschip-12705_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-2817_1','testschip-1290_1'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-3762_1','testschip-1587_5'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-6532_1','testschip-5133_1'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-8702_0','testschip-14181_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-5981_0','testschip-4723_1'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-9057_0','testschip-14377_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-3688_0','testschip-4269_1'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-17119_0','testschip-1587_9'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-9057_1','testschip-5202_2'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-3491_2','testschip-6922_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-8702_1','testschip-1191_1'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-4683_0','testschip-18879_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-7523_0','testschip-4908_12'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-5108_1','testschip-5218_1'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-4723_2','testschip-5218_1'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-4352_0','testschip-4723_3'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-1186_7','testschip-6052_3'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-4843_2','testschip-20371_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-2343_0','testschip-20933_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-19943_0','testschip-5459_1'), 
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-4948_2','testschip-12657_1'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-9416_1','testschip-6409_2'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-6903_0','testschip-8786_0'), 
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-6782_2','testschip-18514_0'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-4955_2','testschip-12657_2'),
                            **calculate_additional_waiting_time(origin_destination_matrix,
                                                                'testschip-2528_1','testschip-9592_0')}
additional_waiting_times['testschip-5108_1']+=np.timedelta64(1,'s')

In [ ]:
#Create vessels
list_of_vessels = []
for loc,info in origin_destination_matrix.iterrows():
    Vessel = type('Vessel', (vessel_.IsVessel,
                             port.HasPortAccess, 
                             port.HasAnchorage, 
                             port.HasTurningBasin, 
                             port.HasTerminal), {})
    if info.origin_node in ['8868178','8866859','8866999']:
        bound = 'outbound'
    else:
        bound = 'inbound'
    if info.trip_id in additional_waiting_times.keys():
        additional_waiting_time = additional_waiting_times[info.trip_id]
    else:
        additional_waiting_time = np.timedelta64(0,'s')
    
    created_vessel = create_vessel(Vessel,
                                   env=env,
                                   name=info['name'],
                                   origin=info['origin_node'],
                                   destination=info['berth_node'][0],
                                   next_destination=np.append([],info['destination_node']),
                                   beam=info['width'],
                                   length=info['length'],
                                   draught=info['draught'],
                                   delta_draught=np.append([],info['(un)loading']),
                                   berthing_time=pd.Timedelta(1,'s'),
                                   unloading_time=np.append([],info['(un)loading time']),
                                   turning_time= [pd.Timedelta(0,'s')], #np.append([],info['turning_time'][0]),
                                   arrival_time=info['arrival']+pd.Timedelta(1,'h'),
                                   terminal_of_call=np.array(['Koole' for i in range(len(list(info['berth_of_call'])))]),
                                   berth_of_call=np.array(info['berth_of_call']),
                                   additional_waiting_time = additional_waiting_time,
                                   bound=bound)
    list_of_vessels.append(created_vessel)

In [ ]:
#Add vessels to the network
for index,vessel in enumerate(list_of_vessels):
    sim.add_vessels(vessel=vessel)

## Generate and assign infrastructure

In [ ]:
Koole_berths = pd.DataFrame({'MBL':[12.65,8.15,8.15,8.15,17,12.65,6.8,15.9,15.9],
                             'Length':[250,253.5,253.5,140,395,250,162,222,253]},
                            index=['Kade11','Kade_G','Kade_H','Binnen10','Buiten10','Buiten9','Binnen7','Buiten7','Buiten6'])
Koole_berths.index.name = 'Berth'

In [ ]:
#(Change infrastructure capacities here)
turning_basin_1 = port.IsTurningBasin(env = env, name = 'Turning Basin 1', information = {'Length': 300})
anchorage_1 = port.IsAnchorage(env = env, name = 'Anchorage 1', capacity = 50)
terminal_1 = port.IsJettyTerminal(env=env,name = 'Koole terminal',type='jetty',information=Koole_berths)

In [ ]:
FG.nodes['anchorage']["Anchorage"] = [anchorage_1]
FG.edges['8866999', '8866859',0]["Terminal"] = {'Koole':terminal_1}
FG.nodes['8866999']["Turning Basin"] = [turning_basin_1]

## Append the tidal restriction to the network

In [ ]:
# #Conversion kn to m/s
knots = 0.51444

# #Restrictions as dictionaries
for node in env.FG.nodes:
    env.FG.nodes[node]['Info'] = {}
    env.FG.nodes[node]['Info']['Vertical tidal restriction'] = {}
env.FG.nodes['8861158']['Info']['Horizontal tidal restriction'] = {}

In [ ]:
network_properties = tidal_window_constructor.NetworkProperties()

### Vertical tidal window

In [ ]:
#According to Port of Rotterdam Policy (Change here the ukc policy)
ukc_p = []
ukc_s = []
fwa = []
        
for index,node in enumerate(env.FG.nodes):
    if node in ['8866969','8866305','8864266','8862925','8864465','S14716_B','S14716_A','8860845']:
        ukc_s.append(0.)
        ukc_p.append(0.1)
        fwa.append(0.01)
    elif node in ['8867547','8867980','8866999']:
        ukc_s.append(1.0)
        ukc_p.append(0.0)
        fwa.append(0.025)
    elif node in ['8866859']:
        ukc_s.append(0.5)
        ukc_p.append(0.0)
        fwa.append(0.0)
    elif node in ['anchorage']:
        ukc_s.append(0.0)
        ukc_p.append(0.0)
        fwa.append(0.0)
    else:
        ukc_s.append(0.0)
        ukc_p.append(0.1)
        fwa.append(0.025)
        
for index,node in enumerate(env.FG.nodes):
    vertical_tidal_window_inputs = []

    #Inbound_Vessels_Condition
    vessel_specification = tidal_window_constructor.vessel_specifications({rule_constructor.vessel_characteristics.min_ge_Draught: 0},
                                                                           'x',rule_constructor.vessel_direction.inbound.value)

    window_specification = tidal_window_constructor.vertical_tidal_window_specifications(ukc_s = ukc_s[index],
                                                                                         ukc_p = ukc_p[index],
                                                                                         fwa = fwa[index],)
    
    vertical_tidal_window_inputs.append(tidal_window_constructor.vertical_tidal_window_input(vessel_specifications = vessel_specification,
                                                                                             window_specifications = window_specification))

    #Outbound_Vessels_Condition
    vessel_specification = tidal_window_constructor.vessel_specifications({rule_constructor.vessel_characteristics.min_ge_Draught: 0},
                                                                           'x',rule_constructor.vessel_direction.outbound.value)

    window_specification = tidal_window_constructor.vertical_tidal_window_specifications(ukc_s = ukc_s[index],
                                                                                         ukc_p = ukc_p[index],
                                                                                         fwa = fwa[index],)

    vertical_tidal_window_inputs.append(tidal_window_constructor.vertical_tidal_window_input(vessel_specifications = vessel_specification,
                                                                                             window_specifications = window_specification))

    network_properties.append_vertical_tidal_restriction_to_network(FG,node,vertical_tidal_window_inputs)

### Horizontal tidal window

In [ ]:
#Port of Rotterdam Policy 3rd Petroleum Harbour
previous_nodes = ['8861716','8861674']
node = '8861158'
next_node = '8867547'
knots = 0.5144444
horizontal_tidal_window_inputs = []
scheurkade_data = hydrodynamic_data.sel({'STATION':'Scheurkade'})
scheurkade_data['TIME'] = scheurkade_data.TIME.values - np.timedelta64(20,'m')

for previous_node in previous_nodes:
    #Inbound_Vessels_Condition1
    vessel_specification = tidal_window_constructor.vessel_specifications({rule_constructor.vessel_characteristics.min_ge_Length: 180,
                                                                           rule_constructor.vessel_characteristics.min_ge_Draught: 11.0, #11.0
                                                                           rule_constructor.vessel_characteristics.max_lt_Draught: 14.3},
                                                                           '(x and x and x)',rule_constructor.vessel_direction.inbound.value)

    window_specification = tidal_window_constructor.horizontal_tidal_window_specifications(tidal_window_constructor.horizontal_tidal_window_method.maximum.value,
                                                                                           {tidal_window_constructor.tidal_period.Flood.value: 2*knots,tidal_window_constructor.tidal_period.Ebb.value: 2*knots})


    horizontal_tidal_window_inputs.append(tidal_window_constructor.horizontal_tidal_window_input(vessel_specifications = vessel_specification,
                                                                                                 window_specifications = window_specification,
                                                                                                 condition = {'Origin': previous_node, 'Destination': next_node},
                                                                                                 data = scheurkade_data))
    #Inbound_Vessels_Condition2
    vessel_specification = tidal_window_constructor.vessel_specifications({rule_constructor.vessel_characteristics.min_ge_Draught: 14.3},
                                                  'x',rule_constructor.vessel_direction.inbound.value)

    window_specification = tidal_window_constructor.horizontal_tidal_window_specifications(tidal_window_constructor.horizontal_tidal_window_method.point_based.value,
                                                                                           {tidal_window_constructor.tidal_period.Flood.value: [1.3*0.5*knots,0.7*0.5*knots],tidal_window_constructor.tidal_period.Ebb.value:tidal_window_constructor.accessibility.inaccessible.value})

    horizontal_tidal_window_inputs.append(tidal_window_constructor.horizontal_tidal_window_input(vessel_specifications = vessel_specification,
                                                                                                 window_specifications = window_specification,
                                                                                                 condition = {'Origin': previous_node, 'Destination': next_node},
                                                                                                 data = scheurkade_data))

    #Outbound_Vessels_Condition1  
    vessel_specification = tidal_window_constructor.vessel_specifications({rule_constructor.vessel_characteristics.min_ge_Length: 200,
                                                                           rule_constructor.vessel_characteristics.min_ge_Draught: 12.0, #12.0
                                                                           rule_constructor.vessel_characteristics.max_lt_Draught: 14.3},
                                                                           '(x and x and x)',rule_constructor.vessel_direction.outbound.value)

    window_specification = tidal_window_constructor.horizontal_tidal_window_specifications(tidal_window_constructor.horizontal_tidal_window_method.maximum.value,
                                                                                           {tidal_window_constructor.tidal_period.Flood.value: 2*knots,tidal_window_constructor.tidal_period.Ebb.value:tidal_window_constructor.accessibility.accessible.value})


    horizontal_tidal_window_inputs.append(tidal_window_constructor.horizontal_tidal_window_input(vessel_specifications = vessel_specification,
                                                                                                 window_specifications = window_specification,
                                                                                                 condition = {'Origin': next_node, 'Destination': previous_node},
                                                                                                 data = scheurkade_data))

    #Outbound_Vessels_Condition2
    vessel_specification = tidal_window_constructor.vessel_specifications({rule_constructor.vessel_characteristics.min_ge_Draught: 14.3}, #14.3
                                                  'x',rule_constructor.vessel_direction.outbound.value)

    window_specification = tidal_window_constructor.horizontal_tidal_window_specifications(tidal_window_constructor.horizontal_tidal_window_method.point_based.value,
                                                                                           {tidal_window_constructor.tidal_period.Flood.value: [1.3*0.5*knots,0.7*0.5*knots],tidal_window_constructor.tidal_period.Ebb.value:tidal_window_constructor.accessibility.inaccessible.value})

    horizontal_tidal_window_inputs.append(tidal_window_constructor.horizontal_tidal_window_input(vessel_specifications = vessel_specification,
                                                                                                 window_specifications = window_specification,
                                                                                                 condition = {'Origin': next_node, 'Destination': previous_node},
                                                                                                 data = scheurkade_data))

network_properties.append_horizontal_tidal_restriction_to_network(FG,node,horizontal_tidal_window_inputs)

## Initiation of the simulation

In [ ]:
t1 = timpie.time()
sim.run()
t2 = timpie.time()
print('total simulation time:', t2-t1)

## Output

In [ ]:
vessels = sim.environment.vessels #extract vessels (entitie) from environment. It collects info while it moves through the network. That info is stored in the log file. The log file has 
env = sim.environment #extract the environment itself

### Vessel

In [ ]:
#Create vessel output dataframe
output_df = pd.DataFrame(columns=['Shipname','Length','Beam','Trip_number','Origin','Destination','Routes','Anchorage_area',
                                  'Waiting_time_in_anchorage','Waiting_time_at_terminal','Turning_basin','Turning_time',
                                  'Terminal_of_call','Berth_of_call','(Un)loading_time','Total_sailing_time','Times',
                                  'Actions','Location','Nodes','Sailing_distance','Speed','Heading','Draught','MBL',
                                  'Net_UKC','Gross_UKC','Ship_related_factors','Water_level','Limiting_current_velocity'])
for vessel in vessels:
    df = pd.DataFrame.from_dict(vessel.log)
    if vessel.output['sailed_routes']:
        output_df = pd.concat([output_df,pd.DataFrame(data=[[vessel.name, #'Shipname'   
                                                             vessel.L, #'Length'
                                                             vessel.B, #'Beam'
                                                             0, #'Trip_number'
                                                             vessel.output['sailed_routes'][0][0], #'Origin'
                                                             vessel.output['sailed_routes'][-1][-1], #'Destination'
                                                             vessel.output['sailed_routes'], #'Routes' 
                                                             vessel.output['visited_anchorages'], #'Anchorage_area'
                                                             vessel.output['waiting_times_in_anchorages'], #'Waiting_time_in_anchorage'
                                                             vessel.output['waiting_times_at_terminals'], #'Waiting_time_in_anchorage'
                                                             vessel.output['visited_turning_basins'], #'Turning_basin'
                                                             vessel.output['turning_times'], #'Turning_time'
                                                             vessel.output['visited_terminals'], #'Terminal_of_call'
                                                             vessel.output['visited_berths'], #'Berth_of_call'
                                                             vessel.output['(un)loading_times'], #'(Un)loading_time'
                                                             np.array([status['sailing_time'] for status in df.Status.to_numpy()]).sum(), #'Total_sailing_time'
                                                             df.Time.to_numpy(), #'Times'
                                                             df.Action.to_numpy(), #'Actions'
                                                             df.Location.to_numpy(), #'Location'
                                                             np.array([status['current_node'] for status in df.Status.to_numpy()]), #'Nodes'
                                                             np.array([status['sailing_distance'] for status in df.Status]), #'Sailing_distance'
                                                             np.array([status['speed'] for status in df.Status.to_numpy()]), #'Speed'
                                                             np.array([status['heading'] for status in df.Status.to_numpy()]), #'Heading'
                                                             np.array([status['draught'] for status in df.Status.to_numpy()]), #'Draught'
                                                             np.array([status['MBL'] for status in df.Status.to_numpy()]), #'MBL'
                                                             np.array([status['net_ukc'] for status in df.Status.to_numpy()]), #'Net_UKC'
                                                             np.array([status['gross_ukc'] for status in df.Status.to_numpy()]), #'Gross_UKC'
                                                             np.array([status['ship_related_ukc_factors'] for status in df.Status.to_numpy()]), #'Ship_related_factors'
                                                             np.array([status['water_level'] for status in df.Status.to_numpy()]), #'Water_level'
                                                             np.array([status['limiting current velocity'] for status in df.Status.to_numpy()])]], #'Limiting_current_velocity'
                                                      columns=output_df.columns)])

In [ ]:
output_df = output_df.reset_index(drop=True)
output_df['Trip_id'] = output_df['Shipname']+'_0'

for name in list(dict.fromkeys(output_df.Shipname)):
    in_df = origin_destination_matrix[origin_destination_matrix.name == name]
    out_df = output_df[output_df.Shipname == name]
    for trip_id,loc in enumerate(out_df.index):
        output_df.loc[loc,'Trip_id'] = in_df.iloc[trip_id].trip_id

In [ ]:
waiting_time_comparison = {}
for loc,trip_id in enumerate(output_df.Trip_id):
    modelled_waiting_time_in_anchorage = output_df.loc[loc].Waiting_time_in_anchorage
    modelled_waiting_time_at_terminal = output_df.loc[loc].Waiting_time_at_terminal

In [ ]:
waiting_time_comparison = {}
for loc,trip_id in enumerate(output_df.Trip_id):
    modelled_waiting_time_in_anchorage = output_df.loc[loc].Waiting_time_in_anchorage
    modelled_waiting_time_at_terminal = output_df.loc[loc].Waiting_time_at_terminal
    if modelled_waiting_time_in_anchorage:
        total_waiting_time = np.sum(list(modelled_waiting_time_in_anchorage[0].values()))
        waiting_for_available_terminal = modelled_waiting_time_in_anchorage[0]['Availability']
        waiting_for_tidal_window = modelled_waiting_time_in_anchorage[0]['Tidal window']
        waiting_due_to_priority = modelled_waiting_time_in_anchorage[0]['Priority']
    elif modelled_waiting_time_at_terminal:
        total_waiting_time = np.sum(list(modelled_waiting_time_at_terminal[0][0].values()))
        waiting_for_available_terminal = modelled_waiting_time_at_terminal[0][0]['Availability']
        waiting_for_tidal_window = modelled_waiting_time_at_terminal[0][0]['Tidal window']
        waiting_due_to_priority =  modelled_waiting_time_at_terminal[0][0]['Priority']
    else:
        total_waiting_time = pd.Timedelta(0,'s')
        waiting_due_to_priority = pd.Timedelta(0,'s')
        waiting_for_tidal_window = pd.Timedelta(0,'s')
        waiting_for_available_terminal = pd.Timedelta(0,'s')
    if modelled_waiting_time_at_terminal:
        total_waiting_time += np.sum(list(modelled_waiting_time_at_terminal[0][-1].values()))
        outbound_waiting_for_available_terminal = modelled_waiting_time_at_terminal[0][-1]['Availability']
        outbound_waiting_for_tidal_window = modelled_waiting_time_at_terminal[0][-1]['Tidal window']
        outbound_waiting_due_to_priority =  modelled_waiting_time_at_terminal[0][-1]['Priority']
    else:
        outbound_waiting_due_to_priority = pd.Timedelta(0,'s')
        outbound_waiting_for_tidal_window = pd.Timedelta(0,'s')
        outbound_waiting_for_available_terminal = pd.Timedelta(0,'s')
        
    
    waiting_time_comparison[trip_id] = [waiting_for_tidal_window,
                                        waiting_for_available_terminal,
                                        waiting_due_to_priority,
                                        outbound_waiting_due_to_priority,
                                        outbound_waiting_for_available_terminal,
                                        outbound_waiting_for_tidal_window,
                                        total_waiting_time,
                                        origin_destination_matrix[origin_destination_matrix.trip_id == trip_id].waiting_time_in_anchorage.to_numpy()[0]]

In [ ]:
comparison_df = pd.DataFrame.from_dict(waiting_time_comparison,orient='index',columns=['Waiting_for_tidal_window_inbound',
                                                                                       'Waiting_for_available_berth_inbound',
                                                                                       'Waiting_due_to_priority_inbound',
                                                                                       'Waiting_due_to_priority_outbound',
                                                                                       'Waiting_for_available_berth_outbound',
                                                                                       'Waiting_for_tidal_window_outbound',
                                                                                       'Modelled_total_waiting_time',
                                                                                       'Observed_total_waiting_time'])
for index in comparison_df[pd.isnull(comparison_df['Observed_total_waiting_time'].to_numpy())].index:
    comparison_df.loc[index,'Observed_total_waiting_time'] = np.timedelta64(0,'s')

In [ ]:
with open(path+'\\04_Output_data'+'\\01_Simulation_results'+'\\model_outcome.pickle','wb') as handle:
    pickle.dump(comparison_df, handle, protocol=pickle.HIGHEST_PROTOCOL)